## Sistema de Recomendação de Receitas (baseado em Tags)

Nesta etapa, foi construído um sistema de recomendação capaz de sugerir receitas a partir das tags fornecidas por um cliente. A proposta consiste em retornar as **5 receitas mais relevantes** com base na **similaridade de conteúdo** (content-based filtering).

A entrada consiste em um conjunto de **5 tags**, e o sistema calcula um **score de compatibilidade** entre as tags informadas e as categorias de cada receita. Os resultados são apresentados para **3 perfis distintos de cliente**.



### Carregamento dos dados

Importamos o dataset `receitas_tratadas.csv` contendo as informações das receitas previamente tratadas. Esse será o conjunto base para a construção do sistema de recomendação.


In [16]:
import pandas as pd
import ast
from sklearn.feature_extraction.text import CountVectorizer

df_receitas = pd.read_csv('../data/processed/receitas_tratadas.csv')

display(df_receitas.head())

 
df_receitas['categories_str'] = df_receitas['categories'].apply(lambda x: ','.join(x) if isinstance(x, list) else '')


# Vetorização binária das categorias
vectorizer = CountVectorizer(tokenizer=lambda x: x.split(','), binary=True)
tag_matrix = vectorizer.fit_transform(df_receitas['categories_str'])

# Converter para DataFrame
tag_df = pd.DataFrame(tag_matrix.toarray(), columns=vectorizer.get_feature_names_out())
tag_df.index = df_receitas.index



,directions,fat,date,categories,calories,desc,protein,rating,title,ingredients,sodium,n_ingredients,n_categories,n_directions
0,"['1. Place the stock, lentils, celery, carrot,...",7.0,2006-09-01 04:00:00+00:00,"['Sandwich', 'Bean', 'Fruit', 'Tomato', 'turke...",426.0,NaN,30.0,2.500,"Lentil, Apple, and Turkey Wrap",['4 cups low-sodium vegetable or chicken stock...,559.0,15,11,3
1,['Combine first 9 ingredients in heavy medium ...,23.0,2004-08-20 04:00:00+00:00,"['Food Processor', 'Onion', 'Pork', 'Bake', 'B...",403.0,This uses the same ingredients found in boudin...,18.0,4.375,Boudin Blanc Terrine with Red Onion Confit,"['1 1/2 cups whipping cream', '2 medium onions...",1439.0,28,11,5
2,['In a large heavy saucepan cook diced fennel ...,7.0,2004-08-20 04:00:00+00:00,"['Soup/Stew', 'Dairy', 'Potato', 'Vegetable', ...",165.0,NaN,6.0,3.750,Potato and Fennel Soup Hodge,"['1 fennel bulb (sometimes called anise), stal...",165.0,6,7,2
3,['Heat oil in heavy large skillet over medium-...,NaN,2009-03-27 04:00:00+00:00,"['Fish', 'Olive', 'Tomato', 'Sauté', 'Low Fat'...",NaN,The Sicilian-style tomato sauce has tons of Me...,NaN,5.000,Mahi-Mahi in Tomato Olive Sauce,"['2 tablespoons extra-virgin olive oil', '1 cu...",NaN,10,17,2
4,['Preheat oven to 350°F. Lightly grease 8x8x2-...,32.0,2004-08-20 04:00:00+00:00,"['Cheese', 'Dairy', 'Pasta', 'Vegetable', 'Sid...",547.0,NaN,20.0,3.125,Spinach Noodle Casserole,"['1 12-ounce package frozen spinach soufflé, t...",452.0,6,11,1


/Users/mobiis/Desktop/Data Scientist/.venv/lib/python3.13/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [17]:
def preprocess_categories(cat_list):
    return [c.strip().lower() for c in cat_list]

df_receitas['categories'] = df_receitas['categories'].apply(ast.literal_eval)
df_receitas['categories'] = df_receitas['categories'].apply(preprocess_categories)


In [18]:
def recomendar_receitas(tags_cliente, df, top_n=5, peso_rating=0.3):
    tags_cliente = [tag.strip().lower() for tag in tags_cliente]

    def calcular_score(categorias, rating):
        similaridade = len(set(tags_cliente) & set(categorias))
        return similaridade + (peso_rating * rating)

    df['score'] = df.apply(lambda row: calcular_score(row['categories'], row['rating']), axis=1)
    
    recomendadas = df[df['score'] > 0].sort_values(by='score', ascending=False)
    
    return recomendadas[['title', 'score', 'rating', 'categories']].head(top_n)



In [19]:
clientes = {
    "Cliente A": ['vegetarian', 'healthy', 'quick', 'onion', 'fruit'],
    "Cliente B": ['gourmet', 'pork', 'cheese', 'butter', 'dinner'],
    "Cliente C": ['vegan', 'low sodium', 'gluten-free', 'salad', 'kid-friendly']
}

for nome, tags in clientes.items():
    print(f"\n Top 5 receitas recomendadas para {nome} (tags: {tags}):")
    display(recomendar_receitas(tags, df_receitas))



 Top 5 receitas recomendadas para Cliente A (tags: ['vegetarian', 'healthy', 'quick', 'onion', 'fruit']):


,title,score,rating,categories
10765,Island Salsa,5.5000,5.000,"[condiment/spread, fruit, onion, no-cook, cock..."
1315,"Minted Orange, Fennel and Red Onion Salad",5.5000,5.000,"[salad, fruit, onion, side, vegetarian, quick ..."
13193,"Mango, Jícama and Corn Salad",5.3125,4.375,"[salad, citrus, fruit, herb, onion, vegetable,..."
3047,Spinach and Sun-Dried Tomato Frittata,5.3125,4.375,"[cheese, dairy, egg, fruit, leafy green, onion..."
14114,Tomato and Mint Salad with Pomegranate Dressing,5.3125,4.375,"[fruit, herb, onion, pepper, tomato, vegetable..."



 Top 5 receitas recomendadas para Cliente B (tags: ['gourmet', 'pork', 'cheese', 'butter', 'dinner']):


,title,score,rating,categories
11961,Chipotle Pork Cheeseburgers,5.3125,4.375,"[cheese, pork, quick & easy, backyard bbq, din..."
11020,Sausage and Broccoli Rabe Torta,5.3125,4.375,"[cheese, dairy, pork, vegetable, bake, dinner,..."
17190,Baked Rigatoni with Pork Ragù and Almonds (Pas...,5.1250,3.750,"[cheese, onion, pasta, pork, tomato, sauté, ma..."
8264,Tortilla Soup with Chiles and Tomatoes,4.5000,5.000,"[cheese, garlic, onion, tomato, dinner, winter..."
1893,"Radicchio with Smoked Mozzarella, Balsamic Vin...",4.5000,5.000,"[cheese, dairy, leafy green, pork, side, roast..."



 Top 5 receitas recomendadas para Cliente C (tags: ['vegan', 'low sodium', 'gluten-free', 'salad', 'kid-friendly']):


,title,score,rating,categories
17125,Citrus Salad with Fried Rosemary and Olives,4.5,5.0,"[salad, olive, side, christmas, vegetarian, lo..."
10661,Frisée and Carrot Ribbon Salad with Za'atar-Le...,4.5,5.0,"[salad, leafy green, low carb, low fat, kid-fr..."
7805,Apple Bok Choy Salad,4.5,5.0,"[salad, onion, kid-friendly, low cal, high fib..."
12772,"Mustard Greens, Roasted Squash, and Hazelnut S...",4.5,5.0,"[salad, leafy green, vegetable, side, roast, t..."
7807,Asian Fruit Salad with Papaya-Mint Sauce,4.5,5.0,"[salad, fruit, dessert, side, low sodium, cant..."


### Interpretação dos Resultados

As recomendações refletem bem os perfis dos clientes com base nas tags fornecidas:

- **Cliente A** buscava opções saudáveis e rápidas. O sistema retornou receitas vegetarianas e frescas, como saladas e pratos leves — todas com alto rating.
- **Cliente B**, com foco em refeições mais elaboradas (gourmet, queijo, porco), recebeu sugestões com perfil mais robusto e calórico, como cheeseburgers e massas com carne.
- **Cliente C**, interessado em restrições alimentares (vegano, sem glúten, baixo sódio), obteve receitas bem alinhadas: leves, com ingredientes naturais e excelente avaliação.

### Conclusão

O sistema de recomendação baseado em tags mostrou-se eficaz ao combinar preferências do usuário com avaliações de outros consumidores. A inclusão do peso no rating ajudou a priorizar receitas bem avaliadas, mesmo quando a correspondência de tags era similar.  

Esse tipo de abordagem é útil em contextos onde há grande variedade de itens e preferências personalizadas — como apps de culinária ou e-commerce de alimentos.


Como próximos passos, seria possível explorar melhorias como:
- Ponderar tags mais relevantes.
- Combinar com sistema de recomendação baseado em notas (rating).
- Usar técnicas de NLP (ex: TF-IDF, embeddings) para capturar semântica entre tags.

